In [79]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math

In [80]:
class data_preprocessing:
    def __init__(self,instance_path):
        self.instance_path=instance_path
        
        self.info, self.flights = self.read_file(f_name=self.instance_path)
        
        self.flights_by_day_dict = self.flights_by_day(flight_list=self.flights)
        self.list_days= self.flights_by_day_dict.keys()
        self.airports_by_area = self.get_airports_by_areas()
        self.list_airports=self.get_list_of_airports()
        self.list_areas=list(self.airports_by_area.keys())
        
        self.number_of_areas,self.starting_airport=int(self.info[0][0]),self.info[0][1]
        self.starting_area=self.associated_area_to_airport(airport=self.starting_airport)
        
    def read_file(self,f_name):
        dist = []
        line_nu = -1
        with open(f_name) as infile:
            for line in infile:
                line_nu += 1
                if line_nu == 0:
                    index = int(line.split()[0]) * 2 + 1
                if line_nu >= index:
                    temp = line.split()
                    temp[2] = int(temp[2])
                    temp[3] = int(temp[3])
                    dist.append(temp)
                else:
                    dist.append(line.split())
            info = dist[:int(dist[0][0])*2+1]
            flights = dist[int(dist[0][0])*2+1:]
        return info, flights
    
    def flights_by_day(self,flight_list):
        # Create an empty dictionary to hold flights organized by day
        flights_by_day = {}

        # Iterate over each flight in the input list
        for flight in flight_list:
            # Extract the day from the flight entry
            day = flight[2]

            # Create a flight entry without the day
            flight_without_day = flight[:2] + flight[3:]

            # Add the flight to the corresponding day in the dictionary
            if day not in flights_by_day:
                flights_by_day[day] = []
            flights_by_day[day].append(flight_without_day)
            
        return flights_by_day
    
    def flights_from_airport(self,flights_by_day, from_airport, considered_day):
        flights_from_airport = []
        for day, flights in flights_by_day.items():
            if day==considered_day:
                for flight in flights:
                    if flight[0] == from_airport:
                        flights_from_airport.append(flight)
                return flights_from_airport
            else:
                return None

    def get_cost(self, day, from_airport, to_airport):
        flights = self.flights_by_day_dict.get(day, [])
        return next(
            (
                flight[2]
                for flight in flights
                if flight[0] == from_airport and flight[1] == to_airport
            ),
            float('inf'),
        )
    
    def get_airports_by_areas(self):
        area_num = int(self.info[0][0])
        return {f"{i}": self.info[2+i * 2] for i in range(0, area_num)}
    
    def get_list_of_airports(self):
        unique_airports = set()

        # Iterate through each sublist and add elements to the set
        for sublist in self.airports_by_area.values():
            for airport in sublist:
                unique_airports.add(airport)
        
        return list(unique_airports)
                    
    def associated_area_to_airport(self,airport):
        return next(
            (
                area
                for area, airports in self.airports_by_area.items()
                if airport in airports
            ),
            "Airport not found",
        ) 
    
    def possible_flights_from_an_airport_at_a_specific_day(self,day,from_airport):
        daily_flights = self.flights_by_day_dict.get(day, [])
        
        flights_from_airport = []
        for flight in daily_flights:
            if flight[0] == from_airport:
                
                destination_area = self.associated_area_to_airport(airport=flight[1])
                if destination_area not in self.visited_areas:
                    flights_from_airport.append(flight)

        return flights_from_airport

In [81]:
Data_Preprocessing=data_preprocessing(instance_path="Flight connections dataset/1.in")

In [82]:
class heuristic_operators:
    def __init__(self):
        pass

    @staticmethod
    def swap(arr, a, b):
        arr[a],arr[b] = arr[b], arr[a]

    @staticmethod
    def reverse(arr, a, b):
        if a > b:
            a, b = b, a
        arr[a:b+1] = arr[a:b+1][::-1]

    @staticmethod
    def insert(arr, a, b):
        temp = arr[a]
        del arr[a]
        arr.insert(b, temp)

    @staticmethod
    def swap_k(arr, a, areas_n):
        s = np.random.randint(1, areas_n, size=a * 2)
        for i in range((a // 2) + 2):
            temp = arr[s[i]]
            arr[s[i]] = arr[s[i + 1]]
            arr[s[i + 1]] = temp
            
    @staticmethod
    def swap_target(arr, from_index, to_index):

        if from_index >= len(arr) or to_index >= len(arr):
            raise IndexError("Target or position is out of range")
        
        arr[from_index], arr[to_index] = arr[to_index], arr[from_index]

In [83]:
class heuristics:
    def __init__(self, data_preprocessing_class):
        self.data = data_preprocessing_class
        
        self.starting_airport = self.data.starting_airport
        self.starting_area=self.data.starting_area
        
        self.total_cost = 0
        
        self.create_initial_random_solution()
        self.create_initial_random_solution()
    
    def cost(self, airport_solution):
        # Calculate the total cost of the provided solution
        total_cost = 0
        for day, (from_airport, to_airport) in enumerate(zip(airport_solution, airport_solution[1:]), start=min(self.data.list_days)):
            total_cost += self.data.get_cost(day, from_airport, to_airport)
        return total_cost
    
    def feasibility(self, solution):
        # Check if the provided solution is feasible
        visited_areas = set()
        for day, (from_airport, to_airport) in enumerate(zip(solution, solution[1:]), start=min(self.data.list_days)):
            area = self.data.associated_area_to_airport(to_airport)
            if area in visited_areas:
                return False
            visited_areas.add(area)
            if self.data.get_cost(day, from_airport, to_airport) == float('inf'):
                return False

        # Check if trip starts from the given city and ends in any city of the starting area
        if solution[0] != self.data.starting_airport or self.data.associated_area_to_airport(solution[-1]) != self.data.starting_area:
            return False

        return len(visited_areas) == self.data.number_of_areas
    
    def create_initial_random_solution(self):
        airport_initial_solution=self.data.list_areas
        
        for k in range(len(airport_initial_solution)):
            if airport_initial_solution[k]==self.starting_area:
                heuristic_operators.swap_target(arr=airport_initial_solution,
                                                from_index=k,
                                                to_index=0)
        airport_initial_solution.append(self.starting_area)
        self.initial_solution=airport_initial_solution
        
    def improve_initial_solution(self,initial_solution):
        LLH_a=lambda : heuristic_operators.swap(arr=solution,a=x,b=y)
        
        while self.feasibility(solution=initial_solution)!=True:
            x=np.random.randint(1,len(self.data.list_days))
            y=np.random.randint(1,len(self.data.list_days))
            
            solution=initial_solution.copy()
            
            LLH_a()

            if self.feasibility(solution=solution):
                return True
        
# Create an instance of the heuristics class
Heuristics = heuristics(data_preprocessing_class=Data_Preprocessing)
#Heuristics.improve_initial_solution(Heuristics.initial_solution)

In [84]:
Heuristics.initial_solution

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '0']